<a href="https://colab.research.google.com/github/alejogiley/ChemGraphs/blob/prototype/notebooks/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash

url='https://raw.githubusercontent.com/alejogiley/ChemGraphs/prototype/datasets/estrogen_receptor_alpha.sdf'
curl $url --output estrogen_receptor_alpha.sdf 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.6M  100 34.6M    0     0  6213k      0  0:00:05  0:00:05 --:--:-- 8092k


In [3]:
%%bash

x86='/usr/lib/x86_64-linux-gnu'
url='https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'

# download & extract
curl -L $url | tar xj lib

# move to python packages directory
mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
mv lib/*.so.* $x86/

# rdkit need libboost
ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3845    0  3845    0     0   4188      0 --:--:-- --:--:-- --:--:--  4183
100 20.2M  100 20.2M    0     0  2955k      0  0:00:07  0:00:07 --:--:-- 4384k
mv: cannot move 'lib/python3.6/site-packages/rdkit' to '/usr/local/lib/python3.6/dist-packages/rdkit': Directory not empty
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libboost_python3.so.1.65.1': File exists


In [4]:
import sys

sys.path.append('/usr/local/lib/python3.6/site-packages')

In [5]:
%%capture

!pip install spektral

In [6]:
import os

import numpy as np
import tensorflow as tf
import scipy.sparse as sp
import tensorflow_probability as tfp

from rdkit import Chem
from rdkit.Chem import AllChem
#from rdkit.Chem.Draw import IPythonConsole
#from rdkit.Chem import Draw

from keras.utils import to_categorical

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import (
    Dense, Input, 
    Activation, Dropout,
    BatchNormalization)

from spektral.data import BatchLoader, Dataset, Graph
from spektral.transforms import LayerPreprocess
from spektral.layers import (
    ECCConv, GCSConv, 
    MinCutPool, GlobalSumPool)

#IPythonConsole.ipython_useSVG=True  #< set this to False if you want PNGs instead of SVGs

In [153]:
tf.config.run_functions_eagerly(True)

In [154]:
def ohc(x, keys):
    maps = dict([(k, v) for k, v in zip(keys, range(len(keys)))])
    return to_categorical(maps[x], num_classes=len(keys))

def get_nodes(mol):
    """
    the atomic numbers in this dataset
    {5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53, 78}
    so the on-hot-encoding would be of length 12
    *this is temporary

    """
    keys = [5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53, 78]

    # nodes = np.concatenate((
    #     np.array([(
    #         ohc(atom.GetAtomicNum()), 
    #         atom.GetDoubleProp("_GasteigerCharge"),
    #         atom.atom.GetDegree())
    #     for atom in mol.GetAtoms()]),
    #     mol.GetConformer().GetPositions()[:,:2]),
    #     axis=1
    # )
    AllChem.ComputeGasteigerCharges(mol)
    
    nodes = np.array([( 
        *ohc(atom.GetAtomicNum(), keys).tolist(),
        atom.GetDoubleProp("_GasteigerCharge"),
        atom.GetDegree())
        for atom in mol.GetAtoms()
    ])

    return nodes

def symmetrize(matrix):
    return matrix \
        + np.transpose(matrix, (1, 0, 2)) \
        - np.diag(matrix.diagonal())

def get_edges(mol):
    """
    Same with nodes, the bond types here are
    {'AROMATIC', 'DOUBLE', 'SINGLE', 'TRIPLE'}
    but the number of classes in rdkit is larger
    *temporary solution

    """
    keys = ['AROMATIC', 'DOUBLE', 'SINGLE', 'TRIPLE']

    natms = mol.GetNumAtoms()
    edges = np.zeros((natms, natms, len(keys)))
    
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edges[i, j] = ohc(str(bond.GetBondType()), keys)
    
    return symmetrize(edges)

def str_is_float(s):
    
    try:
        float(s)
        return True
    
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    
    except (TypeError, ValueError):
        pass
 
    return False

def get_labels(mol, key='IC50 (nM)'):
    """Generate label data for each molecule
    
    "rigth" and "left" indicates whether value is right-censored ">"
    or lef-censored "<" which are reported for concentrations beyond 
    detection limits.
    
    "conc" containts the reported concentration values
    angle brackets are removed and boundary values are saved.
    when conc value is 0, it means metric was not reported.
    
    """
    # read potency metric
    sample = mol.GetPropsAsDict()[key]
    # remove leading and trailing whitespaces
    sample = sample.strip()
        
    # below exp. range
    if "<" in sample: 
        
        left = 1
        right = 0
        
        conc = sample.replace('<', '')
        conc = float(conc)

    # outside exp. range
    elif ">" in sample:
        
        left = 0
        right = 1
        
        conc = sample.replace('>', '')
        conc = float(conc)

    # inside exp. range
    elif str_is_float(sample):
        
        left = 0
        right = 0 
        
        conc = sample
        conc = float(conc)

    # no data provided
    else:
        
        left = 0
        right = 0
        conc = 0.0
    
    return np.array([left, right, conc])

In [155]:
# create instance of sdf reader
#suppl = Chem.SDMolSupplier('estrogen_receptor_alpha.sdf', sanitize=True, strictParsing=True)

# read all molecules besides ones with errors into a list
#mols = [mol for mol in suppl if mol is not None]

# Get nodes
x = [get_nodes(mol) for mol in mols]
    
# Adjacency matrices
a = [Chem.rdmolops.GetAdjacencyMatrix(mol) for mol in mols]

# Edge features: bond types
e = [get_edges(mol) for mol in mols]

# Labels: (rank, IC50s)
# this metric is less reliable than e.g. Kd as 
# it depends on the of the substrates used in 
# the essay and it is cell type dependent.
y = [get_labels(mol) for mol in mols]

In [26]:
class EstrogenDB(Dataset):
    """Dataset from BindingDB
    """
    def __init__(self, 
                 n_samples=1000,
                 dpath=None, 
                 nodes=None, 
                 edges=None,
                 adjcs=None, 
                 feats=None,
                 **kwargs):
        self.n_samples = n_samples
        self.nodes = nodes
        self.edges = edges
        self.adjcs = adjcs
        self.feats = feats
        # dataset to load
        self.dpath = dpath
        
        super().__init__(**kwargs)
	
    @Dataset.path.getter
    def path(self):
	    path = os.path.join(self.dpath, f'EstrogenDB.npz')
	    return '' if not os.path.exists(path) else path
	        
    def read(self):
        # create Graph objects
        data = np.load(
            os.path.join(
                self.dpath, 
                f'EstrogenDB.npz'), 
            allow_pickle=True)
        
        # size = len(data['y'])
        
        output = [
            self.make_graph(
                node=data['x'][i],
                adjc=data['a'][i], 
                edge=data['e'][i],
                feat=data['y'][i])
            for i in range(self.n_samples)
            if data['y'][i][-1] > 0
        ]
        
        self.n_samples = len(output)
        
        return output
    
    def download(self):
        # save graph arrays into directory
        filename = os.path.join(self.dpath, f'EstrogenDB')
        
        np.savez_compressed(
            filename, 
            x=self.nodes, 
            a=self.adjcs, 
            e=self.edges, 
            y=self.feats)
    
    @staticmethod
    def make_graph(node, adjc, edge, feat):
        # The node features
        x = node.astype(float)
        
        # The adjacency matrix
        # convert to scipy.sparse matrix
        a = adjc.astype(int)
        a = sp.csr_matrix(a)
        # check shape (n_nodes, n_nodes)
        assert a.shape[0] == len(node)
        assert a.shape[1] == len(node)
        
        # The labels
        y = feat.astype(float)
        # transform IC50 values
        # into pIC50
        y[-1] = np.log10(y[-1])
        
        # The edge features 
        e = edge.astype(float)
        # check shape (n_nodes, n_nodes, ..)
        assert e.shape[0] == len(node)
        assert e.shape[1] == len(node)
        
        return Graph(x=x, a=a, e=e, y=y)

In [27]:
%%time
url = "/content/"

dataset = EstrogenDB(
    n_samples=2000,
    #nodes=x, edges=e, 
    #adjcs=a, feats=y, 
    dpath=url)

CPU times: user 28min 22s, sys: 53 s, total: 29min 15s
Wall time: 29min 20s


In [28]:
dataset

EstrogenDB(n_graphs=887)

In [29]:
# Transform the adjacency matrix 
# according to ECCConv
dataset.apply(LayerPreprocess(ECCConv))

# randomize indexes
indxs = np.random.permutation(len(dataset))

# split 90%/10%
split = int(0.9 * len(dataset))

# Train/test indexes
trnxs, tesxs = np.split(indxs, [split])

# Dataset partition
train, tests = dataset[trnxs], dataset[tesxs]

In [159]:
# class SimpleDense(Layer):

#   def __init__(self, units=32):
#       super(SimpleDense, self).__init__()
#       self.units = units

#   def build(self, input_shape):
#       self.w = self.add_weight(shape=(input_shape[-1], self.units),
#                                initializer='random_normal',
#                                trainable=True)
#       self.b = self.add_weight(shape=(self.units,),
#                                initializer='random_normal',
#                                trainable=True)

#   def call(self, inputs):
#       return tf.matmul(inputs, self.w) + self.b

# def gcnn_model(nodes_shape, edges_shape, channels, n_layers, n_neurons):
    
#     X = Input(shape=(None, nodes_shape))
#     A = Input(shape=(None, None))
#     E = Input(shape=(None, None, edges_shape))

#     y = ECCConv(n_channels)([X, A, E])
#     y = Activation('relu')(y)
    
#     for i in range(1, n_layers):
#         y = ECCConv(n_channels)([y, A, E])
#         y = BatchNormalization(renorm=True)(y)
#         y = Activation('relu')(y)
    
#     # dense block
#     y = GlobalSumPool()(y)
#     y = Dense(n_neurons)(y)
#     y = Activation('relu')(y)
#     y = Dropout(0.25)(y)
#     y = Dense(1)(y)
    
#     # prediction
#     y = Dense(1)(y)
    
#     return Model(inputs=[X, A, E], outputs=O)


# def msent_loss(y_true, y_pred):
    
#     c_true, c_pred = y_true[:, 0], y_pred[:, 1:]
#     p_true, p_pred = y_true[:, 1], y_pred[:, :1]
    
#     # categorical cross-entropy for classes: 0, 1, 2
#     ent = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  
#     # regression error for pIC50 values
#     mse = tf.keras.losses.MeanSquaredError()
    
#     # return the overal error
#     return tf.reduce_mean(
#         ent(c_true, c_pred) + mse(p_true, p_pred))


# def train_model(dataset, epochs, learning_rate, n_channels, n_layers, n_neurons): 
    
#     # Parameters
#     F = dataset.n_node_features  # Dimension of node features
#     S = dataset.n_edge_features  # Dimension of edge features

#     # Create GCN model
#     model = gcn_model(
#         nodes_shape=F, 
#         edges_shape=S, 
#         n_layers=n_layers, 
#         n_neurons=n_neurons,
#         n_channels=n_channels)
    
#     # Compile GCN
#     model.compile(
#         optimizer=Adam(lr=learning_rate), 
#         #metrics=["mae"],
#         loss=msent_loss)
    
#     # Print network summary
#     model.summary()
    
#     loader = BatchLoader(
#         dataset, 
#         batch_size=batch_size)
    
#     # Trains the model
#     history = model.fit(
#         loader.load(),
#         epochs=epochs,
#         steps_per_epoch=loader.steps_per_epoch)
    
#     return model, history

In [33]:
def tobit_loss(y_true, y_pred, sigma, eps=1e-7):

    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    
    # indicators of left-, right-censoring
    y_lefts = y_true[:, 0]
    y_right = y_true[:, 1]
    y_value = y_true[:, 2]
    
    # normal distribution
    normal = tfp.distributions.Normal(loc=0., scale=1.)
    
    # probability function of normal distribution at point y_value
    prob = normal.prob((y_value - y_pred) / sigma) / sigma
    # probability of point random variable being > than y_value
    right_prob = 1 - normal.cdf((y_value - y_pred) / sigma)
    # probability of random variable being < than y_value
    lefts_prob = normal.cdf((y_value - y_pred) / sigma)
    
    # clip tensor values
    prob = tf.clip_by_value(
        prob, 
        clip_value_min=eps, 
        clip_value_max=1/eps)
    
    right_prob = tf.clip_by_value(
        right_prob, 
        clip_value_min=eps, 
        clip_value_max=1/eps)
        
    left_prob = tf.clip_by_value(
        lefts_prob, 
        clip_value_min=eps, 
        clip_value_max=1/eps)
    
    # logarithm of likelihood
    logp = tf.math.log(prob) * (1 - y_right) * (1 - y_lefts) \
           + tf.math.log(right_prob) * y_right * (1 - y_lefts) \
           + tf.math.log(lefts_prob) * y_lefts * (1 - y_right)
    
    return - tf.reduce_sum(logp)

def mse_loss(y_true, y_pred):

    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # indicators of left-, right-censoring
    y_lefts = y_true[:, 0]
    y_right = y_true[:, 1]
    y_value = y_true[:, 2]

    delta = (y_value - y_pred) #* (1 - y_right) * (1 - y_lefts)
    return tf.reduce_sum(tf.square(delta))

def train_gcnn(dataset, epochs, learning_rate, channels, n_layers, n_neurons): 
    
    # Create GCN model
    model = GCNN(
        channels=channels,
        n_layers=n_layers, 
        n_neurons=n_neurons)
    
    # Loader returns batches of graphs
    # with zero-padding done batch-wise
    loader = BatchLoader(
        dataset, batch_size=batch_size, epochs=epochs)
    
    train_loss = tf.keras.metrics.Mean(name='train_loss')

    # Time-based learning rate schedule
    decay_step = 1.0
    decay_rate = learning_rate / epochs
    learning_rate_fn = tf.keras.optimizers.schedules.InverseTimeDecay(
        learning_rate, decay_step, decay_rate)
    optimizer = SGD(learning_rate==learning_rate_fn)
    
    @tf.function(
        input_signature=loader.tf_signature(), 
        experimental_relax_shapes=True)
    def train_step(inputs, targets):
        with tf.GradientTape() as tape:
            #predictions, sigma = model(inputs, training=True)
            #loss = tobit_loss(targets, predictions, sigma)
            predictions = model(inputs, training=True)
            loss = mse_loss(targets, predictions)
            loss += sum(model.losses)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
        #train_loss(loss)
    
    # Train model
    print("Fitting model")
    model_lss = 0
    for k, batch in enumerate(loader):
        
        #train_loss.reset_states()
        loss = train_step(*batch)
        model_lss += loss.numpy()

        if k % loader.steps_per_epoch == 0:
            model_lss /= loader.steps_per_epoch
            print("Epoch {}. Loss: {}".format(
                k // loader.steps_per_epoch, model_lss))
            model_lss = 0

    return model

class GCNN(tf.keras.Model):
    
    def __init__(self, channels, n_layers, n_neurons, **kwargs):
        super(GCNN, self).__init__()
        
        # initialize dense layers
        self.dense1 = Dense(n_neurons)
        self.dense2 = Dense(1)

        # initialize operations
        self.activation = Activation('relu')
        self.dropout = Dropout(0.15)
        self.pooling = GlobalSumPool()
        self.batchnm = BatchNormalization(renorm=True)
        
        # initialize edge-conditioned convolutional layers
        self.conv1 = ECCConv(channels, activation="relu")
        self.convs = []
        for i in range(1, n_layers):
            self.convs.append(ECCConv(channels, activation="relu"))

        # last layer linear model: y = ax + b
        # self.a = tf.Variable(tf.ones([n_neurons, 1]), trainable=True)
        # self.b = tf.Variable(1., trainable=True)

    def call(self, inputs, **kwargs):
        x, a, e = inputs

        #x = tf.cast(x, tf.float32)
        a = tf.cast(a, tf.float32)
        #e = tf.cast(e, tf.float32)
        
        x = self.conv1([x, a, e])
        #x = self.activation(x)
        
        for conv in self.convs:
            x = conv([x, a, e])
            #x = self.batchnm(x)
            #x = self.activation(x)
            # x = self.dropout(x)
        
        x = self.pooling(x)
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return x
        #return tf.matmul(x, self.a) + self.b
        # return tf.matmul(x, self.a), self.b

In [31]:
epochs = 6  # Number of training epochs
batch_size = 32 # MiniBatch sizes
learning_rate = 1e-3 # Optimizer learning rate

n_layers = 12  # number of ECCConv layers
n_neurons = 8  # number of Dense channels
n_channels = 32  # number of Hidden units

In [34]:
model = train_gcnn(train, epochs, learning_rate, n_channels, n_layers, n_neurons)

Fitting model
Epoch 0. Loss: 20.5051318359375
Epoch 1. Loss: 387.0805334472656
Epoch 2. Loss: 397.7018933105469
Epoch 3. Loss: 392.12903198242185
Epoch 4. Loss: 399.5705310058594
Epoch 5. Loss: 396.4114196777344


In [ ]:
model.a.value(), model.b.value()

In [ ]:
dataset[3]['y']

In [ ]:
print("Testing model")
loader = BatchLoader(tests, batch_size=batch_size, epochs=1, shuffle=False)

model_loss = 0.0
for batch in loader:
    inputs, target = batch
    predictions= model(inputs, training=False)
    print(predictions)
    model_loss += mse_loss(target, predictions)

model_loss /= loader.steps_per_epoch
print("Done. Test loss: {}".format(model_loss))

# print("Testing model")
# loader = BatchLoader(tests, batch_size=batch_size, shuffle=False)

# model_loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)
# print("Done. Test loss: {}".format(model_loss))

In [ ]:
[test.y for test in tests]

In [ ]:
tobit_loss(target, predictions, sigma)

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

prediction = model.predict(loader.load(), steps=loader.steps_per_epoch)

pIC50_true = [tests[i]['y'][1] for i in range(tests.n_graphs)]
class_true = [tests[i]['y'][0] for i in range(tests.n_graphs)] 

pIC50_pred = prediction[:, :1]
class_pred = np.argmax(np.apply_along_axis(softmax, 0, prediction[:, 1:]), axis=1)

In [ ]:
pIC50_true, pIC50_pred

In [ ]:
class_true, class_pred